# Génération d'un catalogue d'évènements

## Introduction

Lecture des fichiers j-shis situés dans le dossier "HazardInputs"

Les données qu'ils contiennent permettent de déterminer:
- la liste des sources de séismes
- la probabilité qu'une source génère un séisme à horizon un an
- la loi de magnitude des séismes
- les coordonnées de la forme géométrique la plus adaptées pour calculer une distance à la surface de rupture

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
from EQCAT.parse_files import parse_quakes
seismes=parse_quakes()

('Total quakes : ', 37224)


37'224 sources de séismes

Elles sont réparties en grandes familles :
- les segments et les ensembles de segments
- les points et les ensembles de points
- les multipoints
- les domaines
- les zones

Pour chaque source un processus de survenance est associé : on peut calculer la probabilité d'occurrence d'un séisme à horizon de temps donné (un an).

Toutes ces infirmations sur les sources segments sont sauvegardées dans le fichier "../Results/sources_summary.csv"

In [4]:
time_lapse=1
out=[]
for code in seismes.keys():
    out+=[(seismes[code].code, seismes[code].__class__.__name__, seismes[code].proc.occ_proba(time_lapse))]
out=pd.DataFrame(out, columns=["code", "source" , "proba1an"])
out.to_csv("../Results/sources_summary.csv", ",")

out.head()

,code,source,proba1an
0,CGR5_25_8697,ZoneEarthquake,0.000802
1,CGR5_25_8696,ZoneEarthquake,0.000802
2,CGR5_25_8695,ZoneEarthquake,0.000801
3,CGR5_25_8694,ZoneEarthquake,0.000800
4,CGR5_25_8693,ZoneEarthquake,0.000799


In [5]:
out.groupby("source").count()

,code,proba1an
source,,
DomainEarthquake,6749,6749
MultiPointsEarthquake,2,2
MultiSegmentEarthquake,32,32
PointsEarthquake,6,6
SegmentEarthquake,380,380
ZoneEarthquake,30055,30055


In [6]:
out.describe()

,proba1an
count,37224.000000
mean,0.002069
std,0.003173
min,0.000000
25%,0.000415
50%,0.000882
75%,0.002128
max,0.042547


Les sources génèrent des évènements qui correspondent aux différentes combinaisons de magnitude et de surfaces de rupture.
La probabilité d'un évènement se calcule comme:
$$\mathbb{P}(event)=\mathbb{P}(source)*\mathbb{P}(magnitude)*\mathbb{P}(shape)$$

## Sources segments

Parameters for seismic activity evaluation and Fault shape of EarthQuake’s Traces Hardly Recognized (EQTHR) from surface evidences

Parameters for seismic activity evaluation

Fault shape (rectangle)

In [8]:
out=[]
for code in seismes.keys():
    if seismes[code].__class__.__name__ == "SegmentEarthquake":
        out+=[(seismes[code].code, seismes[code].proc.__class__.__name__, seismes[code].proc.desc(),
           seismes[code].mag.__class__.__name__, seismes[code].mag.desc(),
           seismes[code].shape.__class__.__name__, seismes[code].shape.desc())]
out=pd.DataFrame(out, columns=["code", "process_class", "process", "magnitude_class", "magnitude", "shape_class", "shape"])
out.to_csv("../Results/segments_summary.csv", ",")
out.head()

,code,process_class,process,magnitude_class,magnitude,shape_class,shape
0,F016021,BrownianPTProcess,"BPT(mu=2000.0,t0=39.0,alph=0.24)",GutenbergRichter,"GR(min=6.38,max=6.62,beta=2.07232658369)",MultiPlaneShape,"MLTPLN(PLN((x=-70.5,y=-95.97,d=2.0)(x=-63.62,y..."
1,F017201,BrownianPTProcess,"BPT(mu=20000.0,t0=7600.0,alph=0.24)",CharacteristicMag,CHAR(mag=6.38),MultiPlaneShape,"MLTPLN(PLN((x=-481.16,y=-25.7,d=2.0)(x=-494.6,..."
2,F007801,PoissonProcess,P(lmbd=8300.0),GutenbergRichter,"GR(min=6.38,max=6.7,beta=2.07232658369)",MultiPlaneShape,"MLTPLN(PLN((x=-381.54,y=-25.7,d=1.0)(x=-406.67..."
3,F007802,PoissonProcess,P(lmbd=5000.0),GutenbergRichter,"GR(min=6.38,max=6.7,beta=2.07232658369)",MultiPlaneShape,"MLTPLN(PLN((x=-384.16,y=-49.49,d=1.0)(x=-413.4..."
4,F007803,BrownianPTProcess,"BPT(mu=3500.0,t0=2400.0,alph=0.24)",GutenbergRichter,"GR(min=6.38,max=6.85,beta=2.07232658369)",MultiPlaneShape,"MLTPLN(PLN((x=-368.08,y=-69.51,d=1.0)(x=-394.4..."


### Survenance

Le processus de survenance est:
- soit un processus de Poisson
- soit un premier passage de mouvement brownien BPT(mu, t0, alpha)

Voir "Parameters for seismic activity evaluation"

In [10]:
pd.crosstab(index=out["process_class"], columns="count")

col_0,count
process_class,
BrownianPTProcess,143
PoissonProcess,237


### Magnitude

Sachant qu'un séisme survient, la loi de magnitude est:

soit une loi de gutemberg richter
$$\mathbb{P}(M \le m) = \frac{\exp(-\beta m_{min}) - \exp(-\beta m)}{\exp(-\beta m_{min}) \exp(-\beta m_{max})} \text{ , } m_{min} \le m \le m_{max}$$
soit une magnitude caractéristique (déterministe)
Voir "Parameters for seismic activity evaluation and Fault shape of EarthQuake’s Traces Hardly Recognized (EQTHR) from surface evidences” et "Fault shape (rectangle)"

In [11]:
pd.crosstab(index=out["magnitude_class"], columns="count")

col_0,count
magnitude_class,
CharacteristicMag,193
GutenbergRichter,187


### Shape

La surface de rupture sachant un séisme est déterministe : un ensemble de rectangles avec chacun 4 points avec les coordonées en 3D. Attention, les longitudes et latitudes, coordonnées sphériques, ont été transformées en coordonnées cartésiennes (unité = km) avec pour centre du repère, le Fundamental Point Azabudai, Minato-ku, Tokyo (lat = 35.658099, lon = 139.741358), le centre du Japanese Geodetic Datum 2000 dans lequel sont exprimés les coordonnées.



In [12]:
out["shape"][0]

'MLTPLN(PLN((x=-70.5,y=-95.97,d=2.0)(x=-63.62,y=-100.04,d=2.0)(x=-70.5,y=-95.97,d=12.0)(x=-63.62,y=-100.04,d=12.0)),PLN((x=-63.73,y=-100.09,d=2.0)(x=-43.73,y=-100.3,d=2.0)(x=-63.73,y=-100.09,d=12.0)(x=-43.73,y=-100.3,d=12.0)))'

Toutes ces infirmations sur les sources segments sont sauvegardées dans le fichier "../Results/segments_summary.csv"



In [13]:
out.head()

,code,process_class,process,magnitude_class,magnitude,shape_class,shape
0,F016021,BrownianPTProcess,"BPT(mu=2000.0,t0=39.0,alph=0.24)",GutenbergRichter,"GR(min=6.38,max=6.62,beta=2.07232658369)",MultiPlaneShape,"MLTPLN(PLN((x=-70.5,y=-95.97,d=2.0)(x=-63.62,y..."
1,F017201,BrownianPTProcess,"BPT(mu=20000.0,t0=7600.0,alph=0.24)",CharacteristicMag,CHAR(mag=6.38),MultiPlaneShape,"MLTPLN(PLN((x=-481.16,y=-25.7,d=2.0)(x=-494.6,..."
2,F007801,PoissonProcess,P(lmbd=8300.0),GutenbergRichter,"GR(min=6.38,max=6.7,beta=2.07232658369)",MultiPlaneShape,"MLTPLN(PLN((x=-381.54,y=-25.7,d=1.0)(x=-406.67..."
3,F007802,PoissonProcess,P(lmbd=5000.0),GutenbergRichter,"GR(min=6.38,max=6.7,beta=2.07232658369)",MultiPlaneShape,"MLTPLN(PLN((x=-384.16,y=-49.49,d=1.0)(x=-413.4..."
4,F007803,BrownianPTProcess,"BPT(mu=3500.0,t0=2400.0,alph=0.24)",GutenbergRichter,"GR(min=6.38,max=6.85,beta=2.07232658369)",MultiPlaneShape,"MLTPLN(PLN((x=-368.08,y=-69.51,d=1.0)(x=-394.4..."


### Catalogue d'évènements

On se donne un horizon de temps (1 an) et un pas de magnitude (0.1) pour générer le catalogue d'évènements.

La probabilité associée à l'évènement est le produit de la probabilité de survenance et de la probabilité de magnitude.

Ces résultats sont sauvegardés dans "../Results/segments_events.csv"

In [14]:
time_step=1
mag_step=0.1
out=[]
for code in seismes.keys():
    if seismes[code].__class__.__name__ == "SegmentEarthquake":
        out+=[seismes[code].events(time_step, mag_step)]

out=pd.concat(out)
out.to_csv("../Results/segments_events.csv", ",")
out.head()

,Code,Magnitude,Proba,Shape
0,F016021,6.48,1.202264e-183,"MLTPLN(PLN((x=-70.5,y=-95.97,d=2.0)(x=-63.62,y..."
1,F016021,6.62,1.314841e-183,"MLTPLN(PLN((x=-70.5,y=-95.97,d=2.0)(x=-63.62,y..."
0,F017201,6.38,5.456936e-08,"MLTPLN(PLN((x=-481.16,y=-25.7,d=2.0)(x=-494.6,..."
0,F007801,6.48,4.651509e-05,"MLTPLN(PLN((x=-381.54,y=-25.7,d=1.0)(x=-406.67..."
1,F007801,6.58,3.780888e-05,"MLTPLN(PLN((x=-381.54,y=-25.7,d=1.0)(x=-406.67..."


In [17]:
out.describe()

,code,process_class,process,magnitude_class,magnitude,shape_class,shape
count,32,32,32,32,32,32,32
unique,32,1,32,1,8,1,32
top,F01770A,MultiSegmentProcess,"MULTI(F012801,F012802)",CharacteristicMag,CHAR(mag=7.3),MultiPlaneShape,"MLTPLN(PLN((x=-83.15,y=78.71,d=5.0)(x=-22.16,y..."
freq,1,32,1,32,8,32,1


## Sources ensemble de segments

Ces sources représentent la survenance simultanée d'un séisme pour plusieurs sources segments.

Voir "HazardInputs/MultiActivity.csv"

In [16]:
out=[]
for code in seismes.keys():
    if seismes[code].__class__.__name__ == "MultiSegmentEarthquake":
        out+=[(seismes[code].code, seismes[code].proc.__class__.__name__, seismes[code].proc.desc(),
           seismes[code].mag.__class__.__name__, seismes[code].mag.desc(),
           seismes[code].shape.__class__.__name__, seismes[code].shape.desc())]
out=pd.DataFrame(out, columns=["code", "process_class", "process", "magnitude_class", "magnitude", "shape_class", "shape"])
out.to_csv("../Results/multi_segments_summary.csv", ",")

out.head()

,code,process_class,process,magnitude_class,magnitude,shape_class,shape
0,F01310A,MultiSegmentProcess,"MULTI(F013101,F013102,F013103)",CharacteristicMag,CHAR(mag=7.3),MultiPlaneShape,"MLTPLN(PLN((x=-805.83,y=-321.92,d=3.0)(x=-813...."
1,F01310B,MultiSegmentProcess,"MULTI(F013101,F013102)",CharacteristicMag,CHAR(mag=7.1),MultiPlaneShape,"MLTPLN(PLN((x=-805.83,y=-321.92,d=3.0)(x=-813...."
2,F01310C,MultiSegmentProcess,"MULTI(F013102,F013103)",CharacteristicMag,CHAR(mag=7.2),MultiPlaneShape,"MLTPLN(PLN((x=-811.97,y=-334.15,d=3.0)(x=-839...."
3,F01770A,MultiSegmentProcess,"MULTI(F017701,F017702)",CharacteristicMag,CHAR(mag=7.3),MultiPlaneShape,"MLTPLN(PLN((x=-451.09,y=-86.52,d=2.0)(x=-496.0..."
4,F01820A,MultiSegmentProcess,"MULTI(F018101,F018201)",CharacteristicMag,CHAR(mag=7.4),MultiPlaneShape,"MLTPLN(PLN((x=-858.17,y=-111.76,d=4.0)(x=-812...."


### Occurrence

La probabilité de survenances simultanées d'un séisme pour les deux sources segment : 
$$\mathbb{P}(segment_{1+2}) = \mathbb{P}(segment_{1}) * \mathbb{P}(segment_{2})$$

### Magnitude

Magnitude caractéristique, déterministe (voir Source segment)

### Shape

Réunion des géométries des segements associés

### Catalogue d'évènements

Résultats dans "Results/multi_segments_events.csv"

In [19]:
time_step=1
mag_step=0.1
out=[]
for code in seismes.keys():
    if seismes[code].__class__.__name__ == "MultiSegmentEarthquake":
        out+=[seismes[code].events(time_step, mag_step)]
out=pd.concat(out)
out.to_csv("../Results/multi_segments_events.csv", ",")

out.head()

,Code,Magnitude,Proba,Shape
0,F01310A,7.3,5.643990e-11,"MLTPLN(PLN((x=-805.83,y=-321.92,d=3.0)(x=-813...."
0,F01310B,7.1,9.530956e-09,"MLTPLN(PLN((x=-805.83,y=-321.92,d=3.0)(x=-813...."
0,F01310C,7.2,1.289337e-05,"MLTPLN(PLN((x=-811.97,y=-334.15,d=3.0)(x=-839...."
0,F01770A,7.3,9.689087e-10,"MLTPLN(PLN((x=-451.09,y=-86.52,d=2.0)(x=-496.0..."
0,F01820A,7.4,1.493966e-07,"MLTPLN(PLN((x=-858.17,y=-111.76,d=4.0)(x=-812...."


## Source points

Parameters for seismic activity evaluation

Fault shape (non-rectangle)

In [21]:
out=[]
for code in seismes.keys():
    if seismes[code].__class__.__name__ == "PointsEarthquake":
        out+=[(seismes[code].code, seismes[code].proc.__class__.__name__, seismes[code].proc.desc(),
           seismes[code].mag.__class__.__name__, seismes[code].mag.desc(),
           seismes[code].shape.__class__.__name__, seismes[code].shape.desc())]
out=pd.DataFrame(out, columns=["code", "process_class", "process", "magnitude_class", "magnitude", "shape_class", "shape"])
out.to_csv("../Results/points_summary.csv", ",")

out.head()

,code,process_class,process,magnitude_class,magnitude,shape_class,shape
0,ATKCH,BrownianPTProcess,"BPT(mu=72.2,t0=13.3,alph=0.28)",CharacteristicMag,CHAR(mag=8.1),MultiPointShape,"MLTPNT(PNT(x=339.2,y=659.93,d=20.0),PNT(x=334...."
1,ANMRO,BrownianPTProcess,"BPT(mu=72.2,t0=43.5,alph=0.28)",CharacteristicMag,CHAR(mag=7.9),MultiPointShape,"MLTPNT(PNT(x=492.02,y=750.44,d=20.0),PNT(x=487..."
2,ATHOP,BrownianPTProcess,"BPT(mu=600.0,t0=5.8,alph=0.24)",CharacteristicMag,CHAR(mag=9.0),MultiPointShape,"MLTPNT(PNT(x=203.7,y=449.77,d=50.0),PNT(x=212...."
3,ATNI1,MultiSegmentProcess,"MULTI(ATKCH,ANMRO)",CharacteristicMag,CHAR(mag=8.3),MultiPointShape,"MLTPNT(PNT(x=355.82,y=650.81,d=18.0),PNT(x=351..."
4,AETRF,BrownianPTProcess,"BPT(mu=72.2,t0=53.2,alph=0.28)",CharacteristicMag,CHAR(mag=8.1),MultiPointShape,"MLTPNT(PNT(x=829.59,y=955.38,d=27.0),PNT(x=824..."


### Survenance

loi de survenance :
- soit brownien
- soit multi (voir source ensemble de segments)

### Magnitude

Magnitude caractéristique, déterministe (voir Source segment)

### Shape

Nuage de points. La distance du séisme à un point est la distance minimale parmi tous les points de la shape.

### Catalogue d'évènements

In [22]:
time_step=1
mag_step=0.1
out=[]
for code in seismes.keys():
    if seismes[code].__class__.__name__ == "PointsEarthquake":
        out+=[seismes[code].events(time_step, mag_step)]
out=pd.concat(out)
out.to_csv("../Results/points_events.csv", ",")

out.head()

,Code,Magnitude,Proba,Shape
0,ATKCH,8.1,9.359456e-11,"MLTPNT(PNT(x=339.2,y=659.93,d=20.0),PNT(x=334...."
0,ANMRO,7.9,8.789382e-03,"MLTPNT(PNT(x=492.02,y=750.44,d=20.0),PNT(x=487..."
0,ATHOP,9.0,0.000000e+00,"MLTPNT(PNT(x=203.7,y=449.77,d=50.0),PNT(x=212...."
0,ATNI1,8.3,8.226383e-13,"MLTPNT(PNT(x=355.82,y=650.81,d=18.0),PNT(x=351..."
0,AETRF,8.1,2.101937e-02,"MLTPNT(PNT(x=829.59,y=955.38,d=27.0),PNT(x=824..."


## Sources multipoints

Parameters for seismic activity evaluation

Fault shape (non-rectangle)

In [23]:
out=[]
for code in seismes.keys():
    if seismes[code].__class__.__name__ == "MultiPointsEarthquake":
        for quake in seismes[code].quakes.values():
            out+=[(seismes[code].code, seismes[code].proc.__class__.__name__, seismes[code].proc.desc(), quake.code,
                   quake.mag.__class__.__name__, quake.mag.desc(), quake.shape.__class__.__name__, quake.shape.desc())]
out=pd.DataFrame(out, columns=["code", "process_class", "process", "code_2", "magnitude_class", "magnitude",
                               "shape_class", "shape"])
out.to_csv("../Results/multi_points_summary.csv", ",")

out.head()

,code,process_class,process,code_2,magnitude_class,magnitude,shape_class,shape
0,ASGMI,BrownianPTProcess,"BPT(mu=302.7,t0=94.0,alph=0.38)",ASG08,CharacteristicMag,CHAR(mag=8.6),MultiPointShape,"MLTPNT(PNT(x=185.45,y=-106.87,d=7.0),PNT(x=183..."
1,ASGMI,BrownianPTProcess,"BPT(mu=302.7,t0=94.0,alph=0.38)",ASG09,CharacteristicMag,CHAR(mag=7.9),MultiPointShape,"MLTPNT(PNT(x=125.64,y=-64.39,d=14.0),PNT(x=125..."
2,ASGMI,BrownianPTProcess,"BPT(mu=302.7,t0=94.0,alph=0.38)",ASG02,CharacteristicMag,CHAR(mag=8.2),MultiPointShape,"MLTPNT(PNT(x=172.98,y=-59.39,d=10.0),PNT(x=172..."
3,ASGMI,BrownianPTProcess,"BPT(mu=302.7,t0=94.0,alph=0.38)",ASG03,CharacteristicMag,CHAR(mag=8.0),MultiPointShape,"MLTPNT(PNT(x=40.8,y=-1.9,d=29.0),PNT(x=40.8,y=..."
4,ASGMI,BrownianPTProcess,"BPT(mu=302.7,t0=94.0,alph=0.38)",ASG01,CharacteristicMag,CHAR(mag=7.9),MultiPointShape,"MLTPNT(PNT(x=40.8,y=-1.9,d=29.0),PNT(x=40.8,y=..."


### Catalogue d'évènements

In [25]:
time_step=1
mag_step=0.1
out=[]
for code in seismes.keys():
    if seismes[code].__class__.__name__ == "MultiPointsEarthquake":
        out+=[seismes[code].events(time_step, mag_step)]
out=pd.concat(out)
out.to_csv("../Results/multi_points_events.csv", ",")

out.head()

,Code,Magnitude,Proba,Shape
0,ASGMI_ASG08,8.6,0.00001,"MLTPNT(PNT(x=185.45,y=-106.87,d=7.0),PNT(x=183..."
0,ASGMI_ASG09,7.9,0.00001,"MLTPNT(PNT(x=125.64,y=-64.39,d=14.0),PNT(x=125..."
0,ASGMI_ASG02,8.2,0.00001,"MLTPNT(PNT(x=172.98,y=-59.39,d=10.0),PNT(x=172..."
0,ASGMI_ASG03,8.0,0.00001,"MLTPNT(PNT(x=40.8,y=-1.9,d=29.0),PNT(x=40.8,y=..."
0,ASGMI_ASG01,7.9,0.00001,"MLTPNT(PNT(x=40.8,y=-1.9,d=29.0),PNT(x=40.8,y=..."


## Sources domaines

Fault shape (discretized rectangular source faults)

Fault shape (discretized rectangular without specified source faults)

Earthquake occurrence frequency data (Earthquakes without specified source faults)

Parameters for seismic activity evaluation

In [3]:
time_step=1
out=pd.DataFrame(columns=["code", "process_class", "process", "code_2", "magnitude", "nb_planes", "shape", "proba_1", "proba_2"])
for q in seismes.values():
    if q.__class__.__name__ == "DomainEarthquake":
        #6749
        proba = q.proc.occ_proba(time_step)
        for mag, p, dom_id in q.mag.mag_probs():
            out=out.append({"code":q.code, "process_class":q.proc.__class__.__name__, "process":q.proc.desc(),
                        "code_2":dom_id, "magnitude":mag, "nb_planes":len(q.dom_dict[dom_id].shape_list),
                            "shape": q.dom_dict[dom_id].desc(), "proba_1":p,
                        "proba_2":1 / float(len(q.dom_dict[dom_id].shape_list))}, ignore_index=True)
out.to_csv("../Results/domain_summary.csv", ",")

out.head()

,code,process_class,process,code_2,magnitude,nb_planes,shape,proba_1,proba_2
0,CPCF_INTRA_13_5633,YearFreqProcess,FRQ(0.00138702),1,7.01,29,"MLTPLN(PLN((x=43.97,y=-123.99,d=86.0)(x=55.1,y...",0.00098,0.034483
1,CPCF_INTRA_13_5633,YearFreqProcess,FRQ(0.00138702),1,7.09,29,"MLTPLN(PLN((x=43.97,y=-123.99,d=86.0)(x=55.1,y...",0.00073,0.034483
2,CPCF_INTRA_13_5633,YearFreqProcess,FRQ(0.00138702),1,7.16,29,"MLTPLN(PLN((x=43.97,y=-123.99,d=86.0)(x=55.1,y...",0.00057,0.034483
3,CPCF_INTRA_13_5633,YearFreqProcess,FRQ(0.00138702),1,7.24,29,"MLTPLN(PLN((x=43.97,y=-123.99,d=86.0)(x=55.1,y...",0.00038,0.034483
4,CPCF_INTRA_13_5633,YearFreqProcess,FRQ(0.00138702),1,7.32,29,"MLTPLN(PLN((x=43.97,y=-123.99,d=86.0)(x=55.1,y...",0.00030,0.034483
